In [5]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input
from datetime import datetime
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import faker
fak = faker.Faker()

In [29]:
err = fak.location_on_land()
err

('32.35126', '-95.30106', 'Tyler', 'US', 'America/Chicago')

In [30]:
fak.location_on_land()[4].split('/')[0]

'Asia'

In [31]:
#1. Load web log file
test_data = pd.read_csv('websitelog.csv')

#2. Data Transformation
first = test_data.columns
columns = ['ip_address','datetime','HTTPResponse','last_status_code',
                'sports_event','interaction_type','view_duration(mins)','Ad_Revenue($)','session_duration(mins)','Latitude','Longitude','country_of_origin','continent','traffic_source',
                'device_type','browser_type','response_time','content_size']
test_data.columns = columns
test_data.loc[len(test_data.index)] = first
test_data = test_data.sort_values(by='datetime')


#2.1 Change data types
test_data[['view_duration(mins)']] = test_data[['view_duration(mins)']].astype('int64')
test_data[['Ad_Revenue($)']] = test_data[['Ad_Revenue($)']].astype('int64')
test_data[['session_duration(mins)']] = test_data[['session_duration(mins)']].astype('int64')


test_data.head()

,ip_address,datetime,HTTPResponse,last_status_code,sports_event,interaction_type,view_duration(mins),Ad_Revenue($),session_duration(mins),Latitude,Longitude,country_of_origin,continent,traffic_source,device_type,browser_type,response_time,content_size
70797,178.120.22.141,2019-01-01,POST /usr HTTP/1.0,202,Boxing,likes,1177,3438,4968,31.39547,101.96857,CR,Europe,organic search,tablet,Edge,552,3324
95961,168.166.249.137,2019-01-01,GET /usr HTTP/1.0,404,Golf,likes,959,4292,109,33.86419,84.63445,TR,Europe,direct,mobile,Explorer,550,2480
81040,140.190.78.20,2019-01-01,GET /usr/register HTTP/1.0,200,Equastrian,comment,1096,4784,3932,14.62072,72.93924,BR,America,direct,Desktop,FireFox,447,3465
46352,168.187.123.51,2019-01-01,POST /usr/register HTTP/1.0,403,Hockey,likes,543,1198,593,7.6,79.41727,DE,Europe,Facebook,Desktop,Safari,647,548
58918,197.196.72.31,2019-01-01,GET /usr/login HTTP/1.0,200,Athletics,share,9,406,4669,-37.32167,-44.31389,BR,America,twitter,laptop,FireFox,363,3154


In [32]:
test_data[['country_of_origin']].value_counts()

country_of_origin
US                   16567
IN                    7261
RU                    5507
DE                    5020
BR                    4166
                     ...  
VU                      91
DJ                      88
AF                      88
DK                      86
GM                      85
Name: count, Length: 134, dtype: int64

In [33]:
#2.1 Change data types
test_data[['view_duration(mins)']] = test_data[['view_duration(mins)']].astype('int64')
test_data[['Ad_Revenue($)']] = test_data[['Ad_Revenue($)']].astype('int64')
test_data[['session_duration(mins)']] = test_data[['session_duration(mins)']].astype('int64')
#test_data[['Latitude']] = test_data[['Latitude']].astype('float64')
#test_data[['Longitude']] = test_data[['Longitude']].astype('float64')


test_grp3 = test_data[['Latitude','Longitude','view_duration(mins)','Ad_Revenue($)','session_duration(mins)']].groupby(['Latitude','Longitude',],as_index=False).sum()
test_grp4 = test_data[['country_of_origin','Latitude','Longitude','view_duration(mins)','Ad_Revenue($)','session_duration(mins)']].groupby(['country_of_origin','Latitude','Longitude',],as_index=False).sum()
test_grp5 = test_data[['continent','Latitude','Longitude','view_duration(mins)','Ad_Revenue($)','session_duration(mins)']].groupby(['continent','Latitude','Longitude'],as_index=False).sum()

In [34]:
test_grp3.tail()

,Latitude,Longitude,view_duration(mins),Ad_Revenue($),session_duration(mins)
94865,68.79833,137.36215,893,235,3711
94866,68.79833,140.73667,412,1553,3042
94867,68.79833,153.10236,291,4305,1502
94868,68.79833,178.00417,1102,3052,2918
94869,46.08333,-91.06177,77,1693,4014


In [35]:
test_grp4.head()

,country_of_origin,Latitude,Longitude,view_duration(mins),Ad_Revenue($),session_duration(mins)
0,AD,-54.8,-120.45794,148,3477,689
1,AD,-32.45242,73.08333,575,4556,1412
2,AD,-32.45242,110.71639,820,2354,1021
3,AD,-32.05251,-71.06616,384,1779,2407
4,AD,-22.9,12.16179,1064,4715,2135


In [36]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/volcano_db.csv",
    encoding="iso-8859-1",
)
df.head()

,Number,Volcano Name,Country,Region,Latitude,Longitude,Elev,Type,Status,Last Known
0,0803-001,Abu,Japan,Honshu-Japan,34.500,131.600,571.0,Shield volcano,Holocene,Unknown
1,1505-096,Acamarachi,Chile,Chile-N,-23.300,-67.620,6046.0,Stratovolcano,Holocene,Unknown
2,1402-08=,Acatenango,Guatemala,Guatemala,14.501,-90.876,3976.0,Stratovolcano,Historical,D1
3,0103-004,Acigol-Nevsehir,Turkey,Turkey,38.570,34.520,1689.0,Maar,Holocene,U
4,1201-04-,Adams,United States,US-Washington,46.206,-121.490,3742.0,Stratovolcano,Tephrochronology,D6


In [37]:
df.dtypes

Number           object
Volcano Name     object
Country          object
Region           object
Latitude        float64
Longitude       float64
Elev            float64
Type             object
Status           object
Last Known       object
dtype: object

In [52]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# frequency of Country
freq = test_grp4
freq = test_grp4.country_of_origin.value_counts().reset_index().rename(columns={"index": "x"})

fre2 = test_grp5
freq2 = test_grp5.continent.value_counts().reset_index().rename(columns={"index": "x"})


data = test_grp3.sample(1000)
# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
           [            None                    , {"type": "bar"}]])

# Add scattergeo globe map of volcano locations
fig.add_trace(
    go.Scattergeo(lat=data["Latitude"],
                  lon=data["Longitude"],
                  mode="markers",
                  hoverinfo="text",
                  showlegend=False,
                  marker=dict(color="crimson", size=4, opacity=0.8)),
    row=1, col=1
)

# Add locations bar chart
fig.add_trace(
    go.Bar(x=freq2["continent"][0:10],y=freq2["count"][0:10], marker=dict(color="crimson"), showlegend=False),
    row=1, col=2
)

# Add locations bar chart
fig.add_trace(
    go.Bar(x=freq["country_of_origin"][0:10],y=freq["count"][0:10], marker=dict(color="crimson"),showlegend=False),
    row=2, col=2
)

# Update geo subplot properties
fig.update_geos(
    projection_type="orthographic",
    landcolor="white",
    oceancolor="MidnightBlue",
    showocean=True,
    lakecolor="LightBlue",
)

# Rotate x-axis labels
fig.update_xaxes(tickangle=45)

# Set theme, margin, and annotation in layout
fig.update_layout(
    margin=dict(r=10, t=25, b=40, l=60),
    title_text='Total count of visits for each country of origin and continent',
    annotations=[
        dict(
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ]
)

fig.show()

In [39]:
freq

,country_of_origin,count
0,US,16412
1,IN,7229
2,RU,5491
3,DE,5008
4,BR,4165
...,...,...
129,VU,91
130,DJ,88
131,AF,88
132,DK,86


In [43]:
freq2

,continent,count
0,Europe,30829
1,America,28837
2,Asia,27787
3,Africa,8673
4,Australia,1217
5,Pacific,546
6,Indian,498
7,Atlantic,213


In [46]:
test_grp3.value_counts().reset_index().rename(columns={"index": "x"}).sample(10)

,Latitude,Longitude,view_duration(mins),Ad_Revenue($),session_duration(mins),count
69359,-13.7804,-79.62603,1181,2363,1880,1
39834,46.18396,75.06583,1195,146,2697,1
17702,37.05944,109.22921,176,2095,522,1
11031,44.27804,103.61667,846,7243,3712,1
30593,55.82885,-58.44417,352,4625,2900,1
9621,43.50891,-77.96389,60,4796,313,1
88051,18.87813,120.82747,986,3603,3439,1
85192,23.0072,-120.45794,1199,363,2929,1
78257,35.06667,139.55722,1047,4086,4495,1
88917,18.03496,39.8302,1246,848,2425,1
